In [ ]:
import cdflib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from kamodo_ccmc.flythrough.utils import ConvertCoord
from sgp4.api import Satrec, jday
from datetime import datetime, timezone, timedelta
import re
from pylab import rcParams
import xarray as xr
from scipy import constants as pc
import pyspedas.cluster as cl
from pytplot import tplot, get_data, time_datetime
import requests
import json
import os
import functions_1 as fn

r_e = 6371.
m0 = pc.physical_constants['atomic mass unit-kilogram relationship'][0]

field_data = xr.open_dataset("../fgm/fgm.nc")
int_list = pd.read_pickle('../mag-field/intersection_list.pkl')

In [ ]:
def sta_epsd_dt_plot(freq_lb, freq_ub, dt_lb, dt_ub, cmp, dir_name, conj_list, label:bool):
    try:
        file_in = os.path.join(dir_name, [i for i in os.listdir(dir_name) if (dt_lb.strftime('%Y%m%d') in i) & ('c2' in i)][0])
        print('File identified')
    except:
        print('CDF does not exist. Downloading files')
        fn.clsta_get('c2', dt_lb.strftime('%Y%m%d'), dt_ub.strftime('%Y%m%d'))
        print('Files downloaded')
        file_in = os.path.join(dir_name, [i for i in os.listdir(dir_name) if (dt_lb.strftime('%Y%m%d') in i) & ('c2' in i)][0])
        print('File identified')
        
    a = cdflib.CDF(file_in)
    dt = pd.to_datetime(cdflib.cdfepoch.to_datetime(a['Time__C2_CP_STA_PSD']))
    freq = a['Frequency__C2_CP_STA_PSD']
    epsd_fv = a.varattsget(6)['FILLVAL']
    epsd_cmp = a['EE_xxyy_sr2__C2_CP_STA_PSD']
    epsd_cmp[epsd_cmp == epsd_fv] = np.nan
    epsd_cmb = np.sum((epsd_cmp**2),axis=2)**0.5

    epsd_xx = pd.DataFrame(data=epsd_cmp[:,:,0], index=dt, columns=freq)
    epsd_yy = pd.DataFrame(data=epsd_cmp[:,:,1], index=dt, columns=freq)
    epsd_tot = pd.DataFrame(data=epsd_cmb, index=dt, columns=freq)

    if cmp == 'xx':
        epsd_df = epsd_xx
        print('Plotting xx component')
    elif cmp == 'yy':
        epsd_df = epsd_yy
        print('Plotting yy component')
    elif cmp == 'tot':
        epsd_df = epsd_tot
        print('Plotting combined components')

    sel_df = epsd_df[ freq[ (freq>freq_lb) & (freq<freq_ub) ]].loc[ dt_lb:dt_ub ]
    sel_conj = conj_list[(conj_list['time'] > dt_lb) & (conj_list['time'] < dt_ub)]
    
    if len(sel_df) == 0:
        print('No data for time range')
        return
    elif sel_df.isnull().values.all():
        print('No data for time range')
        return
    
    # E-field data
    x = sel_df.index.values
    y = sel_df.columns.values
    z = sel_df.values.T
    vmin = np.nanmin(abs((sel_df.replace(0, np.nan).to_numpy()))) * 0.9
    vmax = np.nanmax(sel_df.to_numpy()) * 10

    # plot
    rcParams['figure.figsize'] = 12, 5
    plt.contourf(x,y,z, levels=np.logspace(int(np.floor(np.log10(vmin))-2), int(np.ceil(np.log10(vmax))+2), num=60, base=10.0), norm = colors.LogNorm(vmin = vmin, vmax = vmax), cmap = 'jet')
    
    if label == True:
        for i in range(len(sel_conj)):
            plt.axvline(x = sel_conj.iloc[i]['time'], label = (sel_conj.iloc[i]['id'] + ', ' + str(round(sel_conj.iloc[i]['dist_km'])) + ' km'), ls='--', c='magenta')
    
    clb = plt.colorbar()
    clb.ax.set_ylabel('(mV/m)^2/Hz')
    
    plt.ylabel('Frequency / Hz')
#     plt.legend()
    plt.savefig('../graphics/sta-example.png',dpi=300,bbox_inches='tight')
    plt.show()
    return

In [ ]:
def sta_epsd_sat_plot(cl_id, freq_lb, freq_ub, sat_id, cmp, dir_name, field_data, conj_list, buffer):
    try:
        data_in = conj_list.loc[str(sat_id)]
    except:
        print('Satellite not in database')
    
    dt_lb = data_in['conj_time'] - timedelta(seconds=buffer)
    dt_ub = data_in['conj_time'] + timedelta(seconds=buffer)
    
    try:
        file_in = os.path.join(dir_name, [i for i in os.listdir(dir_name) if (dt_lb.strftime('%Y%m%d') in i) & (cl_id in i)][0])
        print('File identified')
    except:
        print('CDF does not exist. Downloading files')
        clsta_get(cl_id, dt_lb.strftime('%Y%m%d'), dt_ub.strftime('%Y%m%d'))
        print('Files downloaded')
        file_in = os.path.join(dir_name, [i for i in os.listdir(dir_name) if (dt_lb.strftime('%Y%m%d') in i) & (cl_id in i)][0])
        print('File identified')
        
    a = cdflib.CDF(file_in)
    dt = pd.to_datetime(cdflib.cdfepoch.to_datetime(a['Time__' + cl_id + '_CP_STA_PSD']))
    freq = a['Frequency__' + cl_id + '_CP_STA_PSD']
    epsd_fv = a.varattsget(6)['FILLVAL']
    epsd_cmp = a['EE_xxyy_sr2__' + cl_id + '_CP_STA_PSD']
    epsd_cmp[epsd_cmp == epsd_fv] = np.nan
    epsd_cmb = np.sum((epsd_cmp**2),axis=2)**0.5

    epsd_xx = pd.DataFrame(data=epsd_cmp[:,:,0], index=dt, columns=freq)
    epsd_yy = pd.DataFrame(data=epsd_cmp[:,:,1], index=dt, columns=freq)
    epsd_tot = pd.DataFrame(data=epsd_cmb, index=dt, columns=freq)

    if cmp == 'xx':
        epsd_df = epsd_xx
        print('Plotting xx component')
    elif cmp == 'yy':
        epsd_df = epsd_yy
        print('Plotting yy component')
    elif cmp == 'tot':
        epsd_df = epsd_tot
        print('Plotting combined components')

    sel_df = epsd_df[ freq[ (freq>freq_lb) & (freq<freq_ub) ]].loc[ dt_lb:dt_ub ]
    
    if len(sel_df) == 0:
        print('No data for time range')
        return
    elif sel_df.isnull().values.all():
        print('No data for time range')
        return
    
    # E-field data
    x = sel_df.index.values
    y = sel_df.columns.values
    z = sel_df.values.T
    vmin = np.nanmin(abs((sel_df.replace(0, np.nan).to_numpy()))) * 0.9
    vmax = np.nanmax(sel_df.to_numpy()) * 1.1
    
    # B-field data
    field_spec = field_data.sel(t=slice(dt_lb,dt_ub))
    t = field_spec['t'].values
    maxb = np.max(field_spec['bvec'].values)
    fd = field_spec['bvec'].values[:,3] * 1e-9 # flux density in T
    
    n_o = 2.08e5 * 1e6 # in m-3
    n_n = 1.01e4 * 1e6 # in m-3
    n_e = 2.73e4 * 1e6 # in m-3

    omega_pe = (n_e * pc.e**2 / pc.m_e / pc.epsilon_0)**0.5 # electron plasma frequency
    omega_po = (n_o * pc.e**2 / (16*m0) / pc.epsilon_0)**0.5 # O+ ion plasma frequency
    omega_pn = (n_n * pc.e**2 / (7*m0) / pc.epsilon_0)**0.5 # N+ ion plasma frequency

    Omega_e = fd / pc.m_e * pc.e # electron cyclotron frequency
    Omega_o = fd / (16*m0) * pc.e # O+ ion cyclotron frequency
    Omega_n = fd / (7*m0) * pc.e # N+ ion cyclotron frequency

    omega_ii = ( (omega_po**2 * Omega_n**2 + omega_pn**2 * Omega_o**2) / (omega_po**2 + omega_pn**2) )**0.5 # ion-ion hybrid

    omega_lh_o = (Omega_e * Omega_o)**0.5 # lower hybrid frequency for O+
    omega_lh_n = (Omega_e * Omega_n)**0.5 # lower hybrid frequency for N+
    omega_lh = (Omega_e * omega_ii)**0.5 # lower hybrid frequency for combination of both ion species

    # plot
    rcParams['figure.figsize'] = 20, 5
    plt.contourf(x,y,z, levels=np.logspace(int(np.floor(np.log10(vmin))), int(np.ceil(np.log10(vmax))), num=60, base=10.0), norm = colors.LogNorm(vmin = vmin, vmax = vmax), cmap = 'jet')
    clb = plt.colorbar()
    plt.ylabel('Frequency / Hz')
    clb.ax.set_ylabel('(mV/m)^2/Hz')
    plt.axvline(x = data_in['conj_time'], lw=2, ls=':', color='magenta', label='TCA')
    plt.axvline(x = data_in['int_time'], lw=2, ls=':', color='yellow', label='intersection')
    plt.title('3min on 2011-11-29')
    plt.plot(t,omega_ii / (2*np.pi),c='blue',ls='--',label='ion-ion hybrid frequency')
    plt.legend()
    plt.savefig('../graphics/18312_sta_180.png', dpi=300, bbox_inches='tight')
    plt.show()
    return

In [ ]:
def all_epsd_sat_plot(freq_ub, sat_id, dir_name, field_data, conj_list, buffer):
    try:
        data_in = conj_list.loc[str(sat_id)]
    except:
        print('Satellite not in database')
    
    dt_lb = data_in['conj_time'] - timedelta(seconds=buffer)
    dt_ub = data_in['conj_time'] + timedelta(seconds=buffer)
    
    try:
        whi_file_in = os.path.join('../WHISPER/' + dir_name, [i for i in os.listdir('../WHISPER/'+dir_name) if ((dt_lb.strftime('%Y%m%d') in i) & ('c2' in i))][0])
        sta_file_in = os.path.join('../STAFF/' + dir_name, [i for i in os.listdir('../STAFF/'+dir_name) if ((dt_lb.strftime('%Y%m%d') in i) & ('c2' in i))][0])
        print('File identified')
    except:
        print('CDF does not exist')
        
    a = cdflib.CDF(whi_file_in)
    dt = pd.to_datetime(cdflib.cdfepoch.to_datetime(a['time_tags__C2_CP_WHI_NATURAL']))
    whi_freq = np.array([float(i) for i in a['Spectral_Frequencies__C2_CP_WHI_NATURAL'].T])
    epsd_fv = a.varattsget(4)['FILLVAL']
    epsd = a['Electric_Spectral_Power_Density__C2_CP_WHI_NATURAL']
    epsd[epsd == epsd_fv] = np.nan

    whi_epsd_df = pd.DataFrame(data=epsd, index=dt, columns=whi_freq)
    
    a = cdflib.CDF(sta_file_in)
    dt = pd.to_datetime(cdflib.cdfepoch.to_datetime(a['Time__C2_CP_STA_PSD']))
    sta_freq = a['Frequency__C2_CP_STA_PSD']
    epsd_fv = a.varattsget(6)['FILLVAL']
    epsd = a['EE_xxyy_sr2__C2_CP_STA_PSD']
    epsd[epsd == epsd_fv] = np.nan
    epsd = np.sum((epsd**2),axis=2)**0.5
    
    sta_epsd_df = pd.DataFrame(data=epsd, index=dt, columns=sta_freq)

    del epsd, dt, epsd_fv

    whi_sel_df = whi_epsd_df[whi_freq[whi_freq<freq_ub]].loc[ dt_lb:dt_ub ]
    sta_sel_df = sta_epsd_df.loc[ dt_lb:dt_ub ]
    
    if (len(whi_sel_df) == 0) or (len(sta_sel_df) == 0):
        print('No data for time range')
        return
    elif (whi_sel_df.isnull().values.all()) or (sta_sel_df.isnull().values.all()):
        print('No data for time range')
        return
    
    # E-field data
    whi_x = whi_sel_df.index.values
    whi_y = whi_sel_df.columns.values
    whi_z = whi_sel_df.values.T
    whi_vmin = np.nanmin(abs((whi_sel_df.replace(0, np.nan).to_numpy()))) * 0.9
    whi_vmax = np.nanmax(whi_sel_df.to_numpy()) * 10
    
    sta_x = sta_sel_df.index.values
    sta_y = sta_sel_df.columns.values
    sta_z = sta_sel_df.values.T
    sta_vmin = np.nanmin(abs((sta_sel_df.replace(0, np.nan).to_numpy()))) * 0.9
    sta_vmax = np.nanmax(sta_sel_df.to_numpy()) * 10
    
    # B-field data
    field_spec = field_data.sel(t=slice(dt_lb,dt_ub))
    t = field_spec['t'].values
    maxb = np.max(field_spec['bvec'].values)
    fd = field_spec['bvec'].values[:,3] * 1e-9 # flux density in T
    
    n_o = 2.08e5 * 1e6 # in m-3
    n_n = 1.01e4 * 1e6 # in m-3
    n_e = 2.73e4 * 1e6 # in m-3

    omega_pe = (n_e * pc.e**2 / pc.m_e / pc.epsilon_0)**0.5 # electron plasma frequency
    omega_po = (n_o * pc.e**2 / (16*m0) / pc.epsilon_0)**0.5 # O+ ion plasma frequency
    omega_pn = (n_n * pc.e**2 / (7*m0) / pc.epsilon_0)**0.5 # N+ ion plasma frequency

    Omega_e = fd / pc.m_e * pc.e # electron cyclotron frequency
    Omega_o = fd / (16*m0) * pc.e # O+ ion cyclotron frequency
    Omega_n = fd / (7*m0) * pc.e # N+ ion cyclotron frequency

    omega_ii = ( (omega_po**2 * Omega_n**2 + omega_pn**2 * Omega_o**2) / (omega_po**2 + omega_pn**2) )**0.5 # ion-ion hybrid

    omega_lh_o = (Omega_e * Omega_o)**0.5 # lower hybrid frequency for O+
    omega_lh_n = (Omega_e * Omega_n)**0.5 # lower hybrid frequency for N+
    omega_lh = (Omega_e * omega_ii)**0.5 # lower hybrid frequency for combination of both ion species

    # plot
    fig, (ax1, ax2) = plt.subplots(2,1, figsize=(20,12),sharex=True)
    plt.grid(False)
    
    whi_plot = ax1.contourf(whi_x,whi_y,whi_z, levels=np.logspace(int(np.floor(np.log10(whi_vmin))-2), int(np.ceil(np.log10(whi_vmax))+2), num=60, base=10.0), norm = colors.LogNorm(vmin = whi_vmin, vmax = whi_vmax), cmap = 'jet')
    clb_1 = plt.colorbar(whi_plot)
    clb_1.ax.set_ylabel('(V/m)^2/Hz')
    ax1.set_ylabel('Frequency / kHz')
    ax1.axvline(x = data_in['conj_time'], lw=2, ls=':', color='magenta', label='TCA')
    ax1.axvline(x = data_in['int_time'], lw=2, ls=':', color='yellow', label='intersection')
    ax1.plot(t, omega_lh/(2000*np.pi), c='yellow', ls='--', label='lower hybrid frequency')
    ax1.legend()
    ax1.set_title('3min on 2011-11-08')
    
    sta_plot = ax2.contourf(sta_x,sta_y,sta_z, levels=np.logspace(int(np.floor(np.log10(sta_vmin))-2), int(np.ceil(np.log10(sta_vmax))+2), num=60, base=10.0), norm = colors.LogNorm(vmin = sta_vmin, vmax = sta_vmax), cmap = 'jet')
    clb_2 = plt.colorbar(sta_plot)
    clb_2.ax.set_ylabel('(mV/m)^2/Hz')
    ax2.set_ylabel('Frequency / Hz')
    ax2.plot(t,omega_ii / (2*np.pi),c='blue',ls='--', label='ion-ion hybrid frequency')
    ax2.axvline(x = data_in['conj_time'], lw=2, ls=':', color='magenta')
    ax2.axvline(x = data_in['int_time'], lw=2, ls=':', color='yellow')
    ax2.legend()
    
    plt.savefig('../graphics/8197_180.png',dpi=300,bbox_inches='tight')
    plt.show()
    return

In [ ]:
a = pd.read_pickle('../search/all_conj_list.pkl')

In [ ]:
sta_epsd_dt_plot(8, 4000, datetime(2011,11,29,1,2,31), datetime(2011,11,29,1,6,31), 'tot', 'data_psd', a, False)

In [ ]:
all_epsd_sat_plot(30, 8197, 'data_psd', field_data, int_list, 90)

In [ ]:
conj = pd.read_pickle('../WHISPER/whi_conj_200.pkl')
sel_load = conj[ (conj['conj-time'] > datetime(2011,11,1)) & (conj['conj-time'] < datetime(2011,12,1)) & (conj['conj-dist'] < 120) ]

buffer = 180
freq_lb = 10
freq_ub = 4000

for i in sel_load['sat-ID']:
    print(i, pd.to_datetime(sel_load[sel_load['sat-ID']==i]['conj-time'].values[0]))
    dt_lb = (sel_load[sel_load['sat-ID']==i]['conj-time'] - timedelta(seconds=buffer)).array[0]
    dt_ub = (sel_load[sel_load['sat-ID']==i]['conj-time'] + timedelta(seconds=buffer)).array[0]
    sta_epsd_dt_plot('c2', freq_lb, freq_ub, dt_lb, dt_ub, 'tot', 'data_psd', True)

In [ ]:
%%time
sta_epsd_sat_plot('c2', 10, 4000, 18312, 'tot', 'data_psd', field_data, int_list, 90)